# Gunagala package

In [10]:
import numpy as np
from func_lib import zemax_to_array
from astropy import units as u
from astropy.table import Table
from gunagala import optic as optic_pkg
from gunagala import camera as camera_pkg
from gunagala import optical_filter as optical_filter_pkg
from gunagala import psf as psf_pkg
from gunagala import sky as sky_pkg
from gunagala import imager as imager_pkg
import matplotlib 
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt

### Optic class

In [2]:
# Generate Throughput table
num_vals = 11
wl_range = np.linspace(545e-9, 645e-9, num_vals) * 0.9 *  u.meter
relative_throughput = np.ones(num_vals)
throughput = Table([wl_range, relative_throughput], names=("Wavelength", "Throughput"))

In [3]:
# Optic class
aperture = 0.1 * u.meter
focal_length = 3.85 * u.meter
central_obstruction = 0.02 * u.meter

optic = optic_pkg.Optic(aperture, focal_length, throughput, central_obstruction)

### Optical_filter class

In [4]:
# Optical_filter class
num_vals = 101
wl_range = np.linspace(545e-9, 645e-9, num_vals) *  u.meter
relative_throughput = np.ones(num_vals) * 0.99
relative_throughput[0] = 0
relative_throughput[-1] = 0
transmission_table = Table([wl_range, relative_throughput], names=("Wavelength", "Transmission"))

optical_filter = optical_filter_pkg.Filter(transmission=transmission_table)
filters = {"filter_1": optical_filter}

### PSF class

In [17]:
# PSF class
psf_data, metadata = zemax_to_array('Toliman_on-axis_broadband.txt')
psf_sampling = 0.0938 * u.arcsec / u.pixel

pixelated_psf = psf_pkg.PixellatedPSF(psf_data, psf_sampling)

### Camera class

In [18]:
# Generate Quantum Efficiency table
num_vals = 11

QE_range = np.linspace(545, 645, num_vals)
Throughput = np.ones(num_vals) * 0.95
relative_QE = np.ones(num_vals) * 0.95
QE = Table([QE_range, Throughput, relative_QE], names=("Wavelength", "Throughput", "QE"))

In [19]:
# Camera class
fps = 48

bit_depth = 12
full_well = 95e3 * u.electron / u.pixel

gain = 1.29 * u.electron / u.adu
bias = 0 * u.adu / u.pixel # potential

readout_time = 1/fps * u.second # Perturb
pixel_size = 11 * u.micron / u.pix
resolution = [2048, 2048] * u.pix
read_noise = 1.6 * u.electron / u.pix
dark_current = 0.68 * u.electron / (u.second * u.pixel)
minimum_exposure = 1/fps * u.second

camera = camera_pkg.Camera(bit_depth, full_well, gain, bias, readout_time, pixel_size, resolution, read_noise, dark_current, QE, minimum_exposure)

### Sky class

In [20]:
# Sky class
sky = sky_pkg.Simple({"filter_1": 2.18 * u.AB})

### Imager class

In [21]:
# Imager class
imager = imager_pkg.Imager(optic, camera, filters, pixelated_psf, sky)

In [22]:
im = np.asarray(imager.psf.pixellated(size=512))e

In [23]:
im_log = np.log(np.abs(im))

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in log
  """Entry point for launching an IPython kernel.


In [32]:
from matplotlib.colors import LogNorm
plt.imshow(im**0.5)
plt.colorbar()
plt.show()